In [1]:
import sys
sys.path.append(r'c:\ai_project01\Pytorch_Retinaface')
import torch
import cv2
import numpy as np

from models.retinaface import RetinaFace
from data import cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
from utils.box_utils import decode
import os

In [4]:
device = "cuda"
cfg50 = cfg_re50
net = RetinaFace(cfg=cfg50, phase='test').to(device)
pretrained_path = r'c:\ai_project01\Pytorch_Retinaface\weights\Resnet50_Final.pth'
state_dict = torch.load(pretrained_path, map_location=device)
new_state_dict = {}
for k, v in state_dict.items():
    if k.startswith("module."):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v
net.load_state_dict(new_state_dict, strict=True)

C:\ProgramData\anaconda3\envs\yolo_env01\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\ProgramData\anaconda3\envs\yolo_env01\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\User\AppData\Local\Temp\ipykernel_21264\2739834348.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/p

<All keys matched successfully>

In [5]:
net.eval()

RetinaFace(
  (body): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seque

In [9]:
image_dirs = { 'mask_on' : r'C:\ai_project01\mask_images\mask_on', 'no_mask' : r'C:\ai_project01\mask_images\no_mask'}
label_dir = r'C:\ai_project01\labels'
os.makedirs(label_dir, exist_ok=True)

In [8]:
class_ids = {'mask_on' : 0, 'no_mask' : 1}

In [13]:
for mask_status, image_dir in image_dirs.items():
    for img_file in os.listdir(image_dir):
        if img_file.lower().endswith(('png','jpg','jpeg')):
            image_path = os.path.join(image_dir, img_file)
            image_raw = cv2.imread(image_path, cv2.IMREAD_COLOR)
            img = np.float32(image_raw)
            im_height, im_width, _ = img.shape
            scale = torch.Tensor([im_width, im_height, im_width, im_height]).to(device)
            img -= (104, 117, 123)
            img = img.transpose(2, 0, 1)
            img = torch.from_numpy(img).unsqueeze(0).to(device)
            loc, conf, landms = net(img)
            priorbox = PriorBox(cfg50, image_size=(im_height, im_width))
            priors = priorbox.forward().to(device)
            prior_data = priors.data
            boxes = decode(loc.data.squeeze(0), prior_data, cfg50['variance'])
            boxes = boxes * scale
            scores = conf.squeeze(0).data.cpu().numpy()[:,1]
            confidence_threshold = 0.5
            top_indices = np.where(scores > confidence_threshold)[0]
            boxes = boxes[top_indices]
            scores = scores[top_indices]
            dets = np.hstack((boxes.cpu().numpy(), scores[:, np.newaxis])).astype(np.float32, copy=False)
            keep = py_cpu_nms(dets, 0.3)
            dets = dets[keep, :]
            yolo_labels = []
            for b in dets:
                x1, y1, x2, y2 = b[:4]
                x_center = ((x1+x2)/2)/im_width
                y_center = ((y1+y2)/2)/im_height
                bbox_width = (x2-x1)/im_width
                bbox_height = (y2-y1)/im_height
                class_id = class_ids[mask_status]
                yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
            label_path = os.path.join(label_dir, os.path.splitext(img_file)[0] + '.txt')
            with open(label_path, 'w') as f:
                f.write('\n'.join(yolo_labels))
            print(f"Processed: {img_file} -> {label_path} 됐땅")    

Processed: mask_on_00000.png -> C:\ai_project01\labels\mask_on_00000.txt 됐땅
Processed: mask_on_00001.png -> C:\ai_project01\labels\mask_on_00001.txt 됐땅
Processed: mask_on_00002.png -> C:\ai_project01\labels\mask_on_00002.txt 됐땅
Processed: mask_on_00003.png -> C:\ai_project01\labels\mask_on_00003.txt 됐땅
Processed: mask_on_00004.png -> C:\ai_project01\labels\mask_on_00004.txt 됐땅
Processed: mask_on_00005.png -> C:\ai_project01\labels\mask_on_00005.txt 됐땅
Processed: mask_on_00006.png -> C:\ai_project01\labels\mask_on_00006.txt 됐땅
Processed: mask_on_00007.png -> C:\ai_project01\labels\mask_on_00007.txt 됐땅
Processed: mask_on_00008.png -> C:\ai_project01\labels\mask_on_00008.txt 됐땅
Processed: mask_on_00009.png -> C:\ai_project01\labels\mask_on_00009.txt 됐땅
Processed: mask_on_00010.png -> C:\ai_project01\labels\mask_on_00010.txt 됐땅
Processed: mask_on_00011.png -> C:\ai_project01\labels\mask_on_00011.txt 됐땅
Processed: mask_on_00012.png -> C:\ai_project01\labels\mask_on_00012.txt 됐땅
Processed: m